# Project 1

TODO: add small description, imports and comments once the functions are written

In [1]:
#import section
import numpy as np
import math

In [ ]:
# Linear regression using gradient descent
def least_squares_GD(y, tx, initial_w,max_iters, gamma):
    

In [ ]:
#Linear regression using stochastic gradient descent
def least_squares_SGD(y, tx, initial w,max_iters, gamma):
    

In [4]:
#Least squares regression using normal equations
def least_squares(y, tx):
    w_opt = (np.linalg.inv((tx.T).dot(tx)).dot(tx.T)).dot(y)
    y_pred = tx.dot(w_opt)
    return (w_opt,y_pred)

In [ ]:
#Ridge regression using normal equations
def ridge_regression(y, tx, lambda_):
    

In [ ]:
#Logistic regression using gradient descent or SGD
def logistic_regression(y, tx, initial_w,max_iters, gamma):
    

In [ ]:
#Regularized logistic regression using gradient descent or SGD
def reg_logistic_regression(y, tx, lambda_ ,initial_w, max_iters, gamma):
    sig = 1/(1 + math.exp(tx.dot(initial_w)))
    ws = initial_w
    w_len = len(initial_w)
    for i in range max_iters:
        for j in range w_len:
            ws[j] -= lambda_*#derivative of sig?
    w_opt = ws[np.argmin(ws)]
    sig_opt = 1/(1 + math.exp(tx.dot(w_opt)))
    y_pred = np.zeros(len(y))
    for k in range len(y):
        if sig_opt[k] >= 0.5:
            w_pred[k] = 1
    return (w_opt,y_pred)

In [ ]:
# Test: TODO: should plot the results and have test functions to validate 
# the results that we obtain